In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../modules/')

In [ ]:
import pandas as pd
import numpy as np

from preprocess import (load_data, encode_labels, 
                        split_validation, count_atoms, 
                        add_atom_count, add_distance, 
                        center_cos, center_d,
                        decode_labels)
from predict import eval_metric, predict_hgbr
from features import distance
from data_loading import load_data as ld

## Create Simple Model

### Load Training Data

In [ ]:
train, test, coords = load_data()

### Create Train-Validation Split

In [ ]:
x_train, x_val, y_train, y_val = split_validation(train)

### Load GB Inference Model (GBR, XGB, LGB, HGBR)

Starting with HGBR

In [ ]:
y_val_hat = predict_hgbr(x_train, y_train, x_val)

eval_metric(y_val, y_val_hat, x_val.type)

## Feature Creation

### Distance

In [ ]:
d = distance(train)
train_d = train.assign(d=d)

x_d_train, x_d_val, y_train, y_val = split_validation(train_d)
y_d_val_hat = predict_hgbr(x_d_train, y_train, x_d_val)

eval_metric(y_val, y_d_val_hat, x_d_val.type)

### Center Molecule Coords

- [x] Find center of molecule by averaging atom positions
- [x] Subtract centroid coords from molecule coords

In [ ]:
train, test, coords = load_data(True)

In [ ]:
train_dc = train.assign(d=d)

x_dc_train, x_dc_val, y_train, y_val = split_validation(train_dc)
y_dc_val_hat = predict_hgbr(x_dc_train, y_train, x_dc_val)

eval_metric(y_val, y_dc_val_hat, x_dc_val.type)

In [ ]:
atom_freqs = count_atoms(coords)
atom_freqs.head()

train_af = train_dc.merge(atom_freqs, how='left', on='molecule_name')
train_af.head()

In [ ]:
x_af_train, x_af_val, y_af_train, y_af_val = split_validation(train_af)
y_af_hat = predict_hgbr(x_af_train, y_af_train, x_af_val)

eval_metric(y_af_val, y_af_hat, x_af_val.type)

### Using LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
x_af_train.head()

In [ ]:
x_lgb_train = encode_labels(x_af_train)
x_lgb_train.head()

In [ ]:
train_lgb = lgb.Dataset(x_lgb_train, label=y_af_train)

In [ ]:
param = {'objective': 'mae', 'num_leaves': 63}

In [ ]:
bst = lgb.train(param, train_lgb)

In [ ]:
x_lgb_val = encode_labels(x_af_val)
y_lgb_hat = bst.predict(x_lgb_val)

In [ ]:
eval_metric(y_af_val, y_lgb_hat, x_lgb_val.type)

In [ ]:
test.head()

In [ ]:
test_af = test.merge(atom_freqs, how='left', on='molecule_name')
test_af.head()

In [ ]:
test_af = test_af.assign(d=distance(test_af))
test_af.head()

In [ ]:
test_lgb = encode_labels(test_af)
test_lgb.head()

In [ ]:
y_hat = bst.predict(test_lgb)

In [ ]:
y_hat.shape

In [ ]:
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub.head()

In [ ]:
sample_sub.scalar_coupling_constant = y_hat
sample_sub.head()

In [ ]:
sample_sub.to_csv('lgb_sub.csv', index=False)

### Create Features w.r.t. Centroid

- [x] Find angle between atoms w.r.t. centroid
- [x] Distance to centroid and to each other

### Molecule-related Features

- [x] Frequency of each atom in molecule
- [ ] Size of molecule (x,y,z)
- [ ] Weight of molecule

### Molecule Graph Features

- [ ] Use software to infer molecular bonds.
- [ ] Number and types of bonds between atoms.
- [ ] Can find dipole moments, potential energy, magnetic shielding tensor from this?

## Make Preprocess Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

Steps for data transformation:

1. [x] Coordinates from structures.csv
2. [x] Encode types into numerical labels.
3. [x] Encode molecule atoms into columns with atom counts.
4. [x] Center molecule coordinates.
5. [x] Add angle w.r.t. centroid feature. (cos of angle is good enough)
6. Add angle w.r.t. nearest atom feature.
7. [x] Add distance feature.
8. Add columns of bond orders btw atoms.

In [ ]:
train, test, coords = ld()
train, test, encs = encode_labels(train, test)
train, test = add_atom_count(train, test, coords)
train, test = add_distance(train, test)
train, test = center_cos(train, test)
train, test = center_d(train, test)

In [ ]:
train.head()

In [ ]:
train, test = (train.drop(columns='molecule_name'), test.drop(columns='molecule_name'))

In [ ]:
# use lightgbm for prediction
import lightgbm as lgb

x_t, x_v, y_t, y_v = split_validation(train)
train_lgb = lgb.Dataset(x_t, label=y_t)
param = {'objective': 'mae', 'num_leaves': 200}

h = lgb.train(param, train_lgb)
y_v_hat = h.predict(x_v)

In [ ]:
x_v = decode_labels(x_v, encs)

In [ ]:
def metric_by_type(y, y_hat, j_type):
    maes = (y_hat - y).abs().groupby(j_type).mean()
    return np.log(maes.map(lambda x: max(x, 1e-9)))

In [ ]:
metric_by_type(y_v, y_v_hat, x_v.type)

In [ ]:
x_v.reset_index().groupby('type')['index'].count()

In [ ]:
eval_metric(y_v, y_v_hat, x_v.type)

In [ ]:
y_hat = h.predict(test)
sub = pd.read_csv('../data/sample_submission.csv')
sub.loc[:, 'scalar_coupling_constant'] = y_hat
sub.to_csv('../data/lgb_sub_3.csv', index=False)

### Create a Diffrent Hypothesis for each type

In [ ]:
import lightgbm as lgb

In [ ]:
maes_mean = []
for jtype, grp in train.groupby('type'):
    x_t, x_v, y_t, y_v = split_validation(grp)
    train_lgb = lgb.Dataset(x_t, label=y_t)
    param = {'objective': 'mae', 'num_leaves': 200, 'num_iterations': 500}

    h = lgb.train(param, train_lgb)
    y_v_hat = h.predict(x_v)
    
    maes = np.log((y_v_hat - y_v).abs().mean())
    maes_mean.append(maes)
    print(f'Type: {jtype}; Loss: {maes}')
print(f'Metric: {np.mean(maes_mean)}')

In [ ]:
y_hats = []

for jtype in train.type.unique():
    train_j = train.loc[train.type == jtype]
    test_j = test.loc[test.type == jtype]
    
    y_t = train_j.scalar_coupling_constant
    x_t = train_j.drop(columns='scalar_coupling_constant')
    
    train_lgb = lgb.Dataset(x_t, label=y_t)
    param = {'objective': 'mae', 'num_leaves': 200, 'num_iterations': 500}
    
    h = lgb.train(param, train_lgb)
    y_hats.append(test_j.assign(y_hat=h.predict(test_j)))

In [ ]:
y_hat = pd.concat(y_hats).sort_values('id')[['id', 'y_hat']]

In [ ]:
sub = pd.read_csv('../data/sample_submission.csv')
sub.loc[:, 'scalar_coupling_constant'] = y_hat.y_hat
sub.to_csv('../data/lgb_sub_5.csv', index=False)

In [ ]:
train.type.unique()

## TODO

- [x] Fix data loading paths.
- [ ] Implement K-Fold cross-validation. 
- [ ] Add features from artgor kernel.
- [ ] Add angle feature from the other kernel
- [x] Divide the dataset into the different J-coupling types and check the error in each.
- [x] Keep Label Encoder so we can reverse mappings
- [ ] Optimize hyperparams in LightGBM. Try using XGBoost.
- [x] Fix bond.csv and atom.csv indices by making them start at 0.
- [ ] Use bond type and charge features.
- [ ] Create more features from the bond type and charge.
- [ ] Try predicting some of the auxillary data (like Mulliken charges)
- [ ] Reverse label encodings so we can analyze outputs.

In [ ]:
train, test, coords = ld()

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train = pd.read_csv('../data/train.csv', index_col='id')
bonds = pd.read_csv('../data/bonds.csv')
atoms = pd.read_csv('../data/atoms.csv')

In [ ]:
bonds.head()

In [ ]:
train.head()

In [ ]:
atoms.head()

In [ ]:
base_df = train.rename(columns={'atom_index_0': 'a0',
        'atom_index_1': 'a1'})
base_df.head()

In [ ]:
data_df = pd.merge(base_df, atoms, how='left',
        left_on=['molecule_name', 'a0'],
        right_on=['molecule', 'atom_index'])
data_df.head()

In [ ]:
data_df = pd.merge(data_df, atoms, how='left',
        left_on=['molecule_name', 'a1'],
        right_on=['molecule', 'atom_index'],
        suffixes=('_0', '_1'))
data_df.head()

In [ ]:
data_df.drop(columns=['a0', 'a1'], inplace=True)
data_df.head()

In [ ]:
pd.merge(data_df, bonds, how='left',
        left_on=['molecule_name', 'atom_index_0', 'atom_index_1'],
        right_on=['molecule', 'atom_0', 'atom_1']).head()